In [19]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import lightgbm as lgb
import math
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
import warnings
import pdb
warnings.filterwarnings('ignore')
import random
import torch
import torch.nn as nn
import random
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
# from torch.utils.data import TensorDataset,SequentialSampler,RandomSampler,DataLoader
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [20]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)  # 禁止hash随机化
    torch.manual_seed(seed)


set_seed(42)

In [21]:
train = pd.read_csv('./data/train.csv')
train = train.dropna()

test = pd.read_csv('./data/test.csv')
train.info()
test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 141188 entries, 0 to 141187
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   时间         141188 non-null  object 
 1   推料器自动指令    141188 non-null  float64
 2   CO含量       141188 non-null  float64
 3   HCL含量      141188 non-null  float64
 4   NOx含量      141188 non-null  float64
 5   SO2含量      141188 non-null  float64
 6   二次风调门      141188 non-null  float64
 7   二次风量       141188 non-null  float64
 8   给水流量       141188 non-null  float64
 9   炉排实际运行指令   141188 non-null  float64
 10  炉排自动投退信号   141188 non-null  object 
 11  汽包水位       141188 non-null  float64
 12  推料器启停      141188 non-null  object 
 13  推料器自动投退信号  141188 non-null  object 
 14  氧量设定值      141188 non-null  float64
 15  一次风调门      141188 non-null  float64
 16  一次风量       141188 non-null  float64
 17  引风机转速      141188 non-null  float64
 18  主蒸汽流量      141188 non-null  float64
dtypes: float64(15), object(

In [22]:
len(train.columns), len(test.columns)

(19, 18)

In [23]:
df = pd.concat([train, test])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142988 entries, 0 to 1799
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   时间         142988 non-null  object 
 1   推料器自动指令    141188 non-null  float64
 2   CO含量       142988 non-null  float64
 3   HCL含量      142988 non-null  float64
 4   NOx含量      142988 non-null  float64
 5   SO2含量      142988 non-null  float64
 6   二次风调门      142988 non-null  float64
 7   二次风量       142988 non-null  float64
 8   给水流量       142988 non-null  float64
 9   炉排实际运行指令   142988 non-null  float64
 10  炉排自动投退信号   142988 non-null  object 
 11  汽包水位       142988 non-null  float64
 12  推料器启停      142988 non-null  object 
 13  推料器自动投退信号  142988 non-null  object 
 14  氧量设定值      142988 non-null  float64
 15  一次风调门      142988 non-null  float64
 16  一次风量       142988 non-null  float64
 17  引风机转速      142988 non-null  float64
 18  主蒸汽流量      142988 non-null  float64
dtypes: float64(15), object(4)

In [24]:
print('df.shape: ', df.shape)

df.drop(columns=['时间'], inplace=True)


df.shape:  (142988, 19)


In [25]:
df.head(5)

,推料器自动指令,CO含量,HCL含量,NOx含量,SO2含量,二次风调门,二次风量,给水流量,炉排实际运行指令,炉排自动投退信号,汽包水位,推料器启停,推料器自动投退信号,氧量设定值,一次风调门,一次风量,引风机转速,主蒸汽流量
0,60.0,2.6667,2.3519,56.3889,4.8750,19.6343,4330.1270,77.0035,60.0,True,17.1928,False,True,6.0,60.2407,65042.4258,63.1296,61.6909
1,60.0,2.7083,2.3750,56.2407,4.8542,19.6898,4330.1270,77.0324,60.0,True,17.0795,False,True,6.0,60.3333,64419.3359,63.0370,61.3692
2,60.0,2.7361,2.3935,56.2963,4.8125,19.6944,4330.1270,77.0979,60.0,True,17.0938,False,True,6.0,60.4792,64800.8242,63.0532,61.4332
3,60.0,2.7569,2.3519,56.2963,4.7639,19.6713,4582.5752,77.2415,60.0,True,17.2114,False,True,6.0,60.5208,66169.5234,63.0579,61.3655
4,60.0,2.7778,2.3380,56.4259,4.8056,19.6736,4330.1270,77.3591,60.0,True,17.4172,False,True,6.0,60.5370,66389.2734,63.1042,61.4266


In [26]:
# df['gas'] = df.CO_content + df.HCL_content + df.SO2_content + df.NOx_content
#
# # features_names = ['CO_content', 'HCL_content', 'NOx_content', 'SO2_content', 'Primary_air_regulator', 'Primary_air_volume',
# #        'Main_steam_flow_rate_setting', 'Secondary_air_regulator', 'Secondary_air_volume', 'Inducer_fan_speed', 'Pusher_manual_command',
# #        'Pusher_automatic_command', 'Oxygen_setpoint', 'Ladle_water_level', 'Grate_actual_operation_command',
# #        'Grate_manual_command', 'Feedwater_flow']
# features_names = ['Feedwater_flow', 'Oxygen_setpoint', 'Primary_air_volume', 'Main_steam_flow_rate_setting', 'Ladle_water_level']
# for f in tqdm(features_names):
#
#     # 滑动窗口特征
#     window_list = [300, 600, 900]
#     for win_size in window_list:
#         df[f+'_fore_steps_mean_'+str(win_size)] = df[f].rolling(window=win_size, center=False).mean()
#         df[f+'_fore_steps_max_'+str(win_size)] = df[f].rolling(window=win_size, center=False).max()
#         df[f+'_fore_steps_min_'+str(win_size)] = df[f].rolling(window=win_size, center=False).min()
#         df[f+'_fore_steps_std_'+str(win_size)] = df[f].rolling(window=win_size, center=False).std()
#         df[f+'_fore_steps_skew_'+str(win_size)] = df[f].rolling(window=win_size, center=False).skew()
#
#     # n阶差分特征
#     diff_list = list(range(1, 60))
#     for diff_size in diff_list:
#         df[f+'_diff_'+str(diff_size)] = df[f].diff(periods=diff_size)

In [27]:
for col in tqdm(df.columns):
    df[col].fillna(df[col].mean(), inplace=True)

100%|██████████| 18/18 [00:00<00:00, 347.30it/s]


In [28]:
df['炉排自动投退信号'] = df['炉排自动投退信号'].map(int)
df['推料器启停'] = df['推料器启停'].map(int)
df['推料器自动投退信号'] = df['推料器自动投退信号'].map(int)




In [29]:

# 3、内存压缩
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
                start_mem - end_mem) / start_mem))
    return df


# 压缩使用内存
# 由于数据比较大，所以合理的压缩内存节省空间尤为的重要
# 使用reduce_mem_usage函数可以压缩近70%的内存占有。
df = reduce_mem_usage(df)

100%|██████████| 18/18 [00:00<00:00, 328.36it/s]


Mem. usage decreased to  5.86 Mb (71.7% reduction)


In [30]:
print('--------------------data describe: -----------------------------------')
df.info()
df_test = df.tail(1800)
df_train = df.iloc[:-1800, :]
df_train.reset_index(drop=True,inplace=True)

df_valid = df_train.iloc[-1800:, :]
df_train = df_train.iloc[:-1800, :]

df_valid.reset_index(drop=True, inplace=True)

--------------------data describe: -----------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 142988 entries, 0 to 1799
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   推料器自动指令    142988 non-null  float16
 1   CO含量       142988 non-null  float16
 2   HCL含量      142988 non-null  float16
 3   NOx含量      142988 non-null  float16
 4   SO2含量      142988 non-null  float16
 5   二次风调门      142988 non-null  float16
 6   二次风量       142988 non-null  float16
 7   给水流量       142988 non-null  float16
 8   炉排实际运行指令   142988 non-null  float16
 9   炉排自动投退信号   142988 non-null  int8   
 10  汽包水位       142988 non-null  float16
 11  推料器启停      142988 non-null  int8   
 12  推料器自动投退信号  142988 non-null  int8   
 13  氧量设定值      142988 non-null  float16
 14  一次风调门      142988 non-null  float16
 15  一次风量       142988 non-null  float32
 16  引风机转速      142988 non-null  float16
 17  主蒸汽流量      142988 non-null  float16


In [31]:
print('df_train.shape: ', df_train.shape)
print('df_valid.shape: ', df_valid.shape)
print('df_test.shape: ', df_test.shape)

df_train.shape:  (139388, 18)
df_valid.shape:  (1800, 18)
df_test.shape:  (1800, 18)


In [32]:
df_scale = df.iloc[:-1800, :]

In [33]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler = scaler.fit(df)
df_for_training_scaled = scaler.transform(df_train)
df_for_validing_scaled = scaler.transform(df_valid)
# df_for_testing_scaled=scaler.transform(df_test)

In [34]:
df_for_training_scaled

,推料器自动指令,CO含量,HCL含量,NOx含量,SO2含量,二次风调门,二次风量,给水流量,炉排实际运行指令,炉排自动投退信号,汽包水位,推料器启停,推料器自动投退信号,氧量设定值,一次风调门,一次风量,引风机转速,主蒸汽流量
0,60.00000,2.666016,2.351562,56.37500,4.875000,19.640625,4332.0,77.0000,60.0000,1,17.187500,0,1,6.0,60.25000,65042.425781,63.12500,61.6875
1,60.00000,2.708984,2.375000,56.25000,4.855469,19.687500,4332.0,77.0625,60.0000,1,17.078125,0,1,6.0,60.34375,64419.335938,63.03125,61.3750
2,60.00000,2.736328,2.392578,56.28125,4.812500,19.687500,4332.0,77.1250,60.0000,1,17.093750,0,1,6.0,60.46875,64800.824219,63.06250,61.4375
3,60.00000,2.757812,2.351562,56.28125,4.765625,19.671875,4584.0,77.2500,60.0000,1,17.218750,0,1,6.0,60.53125,66169.523438,63.06250,61.3750
4,60.00000,2.777344,2.337891,56.43750,4.804688,19.671875,4332.0,77.3750,60.0000,1,17.421875,0,1,6.0,60.53125,66389.273438,63.09375,61.4375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139383,52.37500,-0.020599,8.289062,49.43750,23.281250,21.375000,6064.0,88.7500,42.9375,1,7.757812,1,1,6.0,60.75000,64566.992188,63.71875,69.6250
139384,52.37500,-0.117920,8.265625,49.28125,23.984375,21.296875,6064.0,88.6250,42.9375,1,7.683594,1,1,6.0,60.68750,64454.105469,63.96875,69.5625
139385,52.40625,-0.159546,8.242188,49.21875,23.968750,21.234375,6064.0,88.5000,42.9375,1,7.898438,1,1,6.0,60.50000,64044.316406,64.18750,69.8750
139386,52.40625,-0.173340,8.234375,49.21875,23.953125,21.281250,5872.0,88.3750,42.9375,1,8.039062,1,1,6.0,60.43750,64210.289062,64.37500,69.7500


In [35]:
def createXY(dataset,n_past):
  dataX = []
  dataY = []
  for i in range(n_past, len(dataset)):
      dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
      dataY.append(dataset[i,0])
  return np.array(dataX),np.array(dataY)


SEQ_LEN = 300 # 即时间窗口


trainX, trainY=createXY(df_for_training_scaled,SEQ_LEN)
validX, validY=createXY(df_for_validing_scaled,SEQ_LEN)
# testX, testY=createXY(df_for_testing_scaled,SEQ_LEN)

In [36]:
print('构建时间序列特征')
print('trainX.shape, trainY.shape: ', trainX.shape, trainY.shape)
print('validX.shape, validY.shape: ', validX.shape, validY.shape)
# print('testX.shape, testY.shape: ', testX.shape, testY.shape)

构建时间序列特征
trainX.shape, trainY.shape:  (139088, 300, 18) (139088,)
validX.shape, validY.shape:  (1500, 300, 18) (1500,)


In [37]:
class My_Dataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        sample = dict(input=torch.FloatTensor(self.data[idx]),
                      label=torch.FloatTensor(np.array(self.labels[idx])))

        return sample


In [38]:
def create_data_loader(data=None, labels=None, batch_size=32):
    ds=My_Dataset(
        data = data,
        labels = labels)
    return DataLoader(ds, batch_size=batch_size, shuffle=False)
    # if test_mode:
    #     return DataLoader(ds, batch_size=batch_size, shuffle=False)
    # else:
    #     return DataLoader(ds, batch_size=batch_size, shuffle=True)

In [39]:
train_data_loader = create_data_loader(data=trainX,
                                       labels=trainY)

val_data_loader = create_data_loader(data=validX,
                                       labels=validY)

In [40]:
len(train_data_loader)

4347

In [41]:
data = next(iter(train_data_loader))
data.keys()
print(data['input'].shape)
print(data['label'].shape)

torch.Size([32, 300, 18])
torch.Size([32])


In [42]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('use device: ', device)

use device:  cuda:0


In [43]:
class Time_Model(nn.Module):

    def __init__(self):
        super().__init__()

        self.hidden_size = 256
        self.num_layers = 1
        self.bidirectional = True
        self.lstm = nn.LSTM(input_size=18,
                             hidden_size=self.hidden_size,
                             num_layers=self.num_layers,
                             bidirectional=self.bidirectional,
                             batch_first=True)

        self.fc = nn.Linear(self.hidden_size*2, 1)


    def forward(self, x):
        device = x.device
        batch_size, seq_len, emb_dim = x.shape
        # 初始化：双向就乘2
        h0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size).to(device)
        c0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size).to(device)

        x, (_, _) = self.lstm(x, (h0, c0))
        x = x[:,-1,:] # 取最后一个时间步
        x = self.fc(x)

        return x


# class Time_Model(nn.Module):
#
#     def __init__(self):
#         super().__init__()
#
#         self.hidden_size = 256
#         self.input_size = 393
#         self.num_layers = 1
#         self.bidirectional = True
#         # self.lstm = nn.LSTM(input_size=22,
#         #                      hidden_size=self.hidden_size,
#         #                      num_layers=self.num_layers,
#         #                      bidirectional=self.bidirectional,
#         #                      batch_first=True)
#
#         self.seq_emb = nn.Sequential(
#             nn.Linear(self.input_size, self.input_size),
#             nn.LayerNorm(self.input_size),
#             nn.GELU(),
#             nn.Dropout(0.1),
#         )
#
#         self.lstm1 = nn.LSTM(self.input_size, self.hidden_size//2, dropout=0.1, batch_first=True, bidirectional=True)
#         self.lstm2 = nn.LSTM(self.hidden_size//2 * 2, self.hidden_size//4, dropout=0.1, batch_first=True, bidirectional=True)
#         self.lstm3 = nn.LSTM(self.hidden_size//4 * 2, self.hidden_size//8, dropout=0.1, batch_first=True, bidirectional=True)
#
#
#         self.head = nn.Sequential(
#             # nn.Linear(self.hidden_size//8 * 2, self.hidden_size//8 * 2),
#             nn.LayerNorm(self.hidden_size//8 * 2),
#             nn.GELU(),
#             #nn.Dropout(0.),
#             nn.Linear(self.hidden_size//8 * 2, 1))
#
#
#     def forward(self, x):
#         device = x.device
#         batch_size, seq_len, emb_dim = x.shape
#         x = self.seq_emb(x)
#
#
#         # 初始化：双向就乘2
#         h0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//2).to(device)
#         c0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//2).to(device)
#         x, (_, _) = self.lstm1(x, (h0, c0))
#
#          # 初始化：双向就乘2
#         h0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//4).to(device)
#         c0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//4).to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#
#         # 初始化：双向就乘2
#         h0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//8).to(device)
#         c0 = torch.randn(self.num_layers * 2, batch_size, self.hidden_size//8).to(device)
#         x, (_, _) = self.lstm3(x, (h0, c0))
#
#
#         x = x[:,-1,:] # 取最后一个时间步
#         x = self.head(x)
#
#         return x


# self.seq_emb = nn.Sequential(
#             nn.Linear(len(cfg.cont_seq_cols), self.hidden_size),
#             nn.LayerNorm(self.hidden_size),
#             nn.GELU(),
#             #nn.Dropout(0.1),
#         )
#         self.lstm1 = nn.LSTM(self.hidden_size, self.hidden_size//2, dropout=0.1, batch_first=True, bidirectional=True)
#         self.lstm2 = nn.LSTM(self.hidden_size//2 * 2, self.hidden_size//4, dropout=0.1, batch_first=True, bidirectional=True)
#         self.lstm3 = nn.LSTM(self.hidden_size//4 * 2, self.hidden_size//8, dropout=0.1, batch_first=True, bidirectional=True)
#         self.head = nn.Sequential(
#             # nn.Linear(self.hidden_size//8 * 2, self.hidden_size//8 * 2),
#             nn.LayerNorm(self.hidden_size//8 * 2),
#             nn.GELU(),
#             #nn.Dropout(0.),
#             nn.Linear(self.hidden_size//8 * 2, 1),

# class Time_Model(nn.Module):
#
#     def __init__(self, input_dim=22, lstm_dim=256,
#                      dense_dim=256, logit_dim=256, num_classes=1):
#         super().__init__()
#
#         self.mlp = nn.Sequential(
#         nn.Linear(input_dim, dense_dim // 2),
#         nn.ReLU(),
#         nn.Linear(dense_dim // 2, dense_dim),
#         nn.ReLU())
#
#         self.lstm = nn.LSTM(dense_dim, lstm_dim, batch_first=True, bidirectional=True)
#
#         self.logits = nn.Sequential(
#             nn.Linear(lstm_dim * 2, logit_dim),
#             nn.ReLU(),
#             nn.Linear(logit_dim, num_classes),
#         )
#
#     def forward(self, x):
#         features = self.mlp(x)
#         features, _ = self.lstm(features)
#         features = features[:,-1,:]
#         pred = self.logits(features)
#
#         return pred

In [44]:
model = Time_Model()
model = model.to(device)

In [45]:
# MSE开方就是RMSE了，所以损失可以直接用来作为评估指标哦
loss_fn = torch.nn.MSELoss(reduction='mean').to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
EPOCHS = 32

In [46]:
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []

    pred_list = []
    target_list = []

    for inputs in tqdm(data_loader):
        targets = inputs["label"].to(device)
        x = inputs['input'].to(device)
        outputs = model(x)
        preds = outputs


        pred_list.extend(preds.cpu().detach().numpy().tolist())
        target_list.extend(targets.cpu().detach().numpy().tolist())


        loss = loss_fn(outputs, targets)

        losses.append(loss.item())
        loss.backward()



        # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()
        # scheduler.step()

    mean_loss = np.mean(losses)


    return mean_loss

In [47]:
def eval_model(model, data_loader, loss_fn, device):
    model = model.eval() # 验证预测模式
    losses = []
    pred_list = []
    target_list = []

    with torch.no_grad():
        for inputs in tqdm(data_loader):
            targets = inputs["label"].to(device)
            x = inputs['input'].to(device)
            outputs = model(x)
            preds = outputs


            pred_list.extend(preds.cpu().detach().numpy().tolist())
            target_list.extend(targets.cpu().detach().numpy().tolist())


            loss = loss_fn(outputs, targets)

            losses.append(loss.item())



    mean_loss = np.mean(losses)

    return mean_loss

In [48]:

from collections import defaultdict
history = defaultdict(list) # 记录10轮loss和acc
best_mse = float('inf')


# -------------------控制早停--------------
early_stop_epochs = 2
no_improve_epochs = 0


for epoch in range(EPOCHS):

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_mse = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
    )

    print(f'train_mse : {train_mse } \n ')

    val_mse = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device
    )

    print(f'val_mse : {val_mse } \n ')
    print()

    history['train_mse '].append(train_mse)
    history['val_mse '].append(val_mse)


    if val_mse  < best_mse :
        print('best model saved!!!!!!!!!!!!!')
        torch.save(model.state_dict(), f'./save model/best_model.pt')
        best_mse  = val_mse

        no_improve_epochs = 0

    else:
        no_improve_epochs += 1



    if no_improve_epochs == early_stop_epochs:
        print('no improve score !!! stop train !!!')
        break

Epoch 1/32
----------
train_mse : 477.2356516457657 
 
val_mse : 20.372872119254254 
 

best model saved!!!!!!!!!!!!!
Epoch 2/32
----------
train_mse : 60.91180114072028 
 
val_mse : 19.390344768128497 
 

best model saved!!!!!!!!!!!!!
Epoch 3/32
----------
train_mse : 59.92571990566608 
 
val_mse : 18.708686474789964 
 

best model saved!!!!!!!!!!!!!
Epoch 4/32
----------
train_mse : 59.70259783033574 
 
val_mse : 18.7229297027943 
 

Epoch 5/32
----------
train_mse : 59.43160635266862 
 
val_mse : 18.62682611003835 
 

best model saved!!!!!!!!!!!!!
Epoch 6/32
----------
train_mse : 59.31163025796859 
 
val_mse : 18.40085232638298 
 

best model saved!!!!!!!!!!!!!
Epoch 7/32
----------
train_mse : 59.15558463237513 
 
val_mse : 18.430848034772467 
 

Epoch 8/32
----------


 56%|█████▋    | 2454/4347 [01:25<01:05, 28.80it/s]


KeyboardInterrupt: 

In [49]:
df_test


,推料器自动指令,CO含量,HCL含量,NOx含量,SO2含量,二次风调门,二次风量,给水流量,炉排实际运行指令,炉排自动投退信号,汽包水位,推料器启停,推料器自动投退信号,氧量设定值,一次风调门,一次风量,引风机转速,主蒸汽流量
0,69.25,11.078125,8.820312,50.34375,5.296875,19.234375,4900.0,79.6250,46.46875,0,2.798828,1,0,6.0,65.0625,68996.023438,66.5625,58.03125
1,69.25,11.031250,8.851562,50.34375,5.277344,19.234375,4664.0,79.6875,46.46875,0,2.857422,1,0,6.0,65.0625,68996.023438,66.5625,58.31250
2,69.25,11.039062,8.867188,50.46875,5.214844,19.203125,4664.0,79.7500,46.46875,0,2.867188,1,0,6.0,65.0625,69109.601562,66.6250,58.28125
3,69.25,11.085938,8.796875,50.59375,5.257812,19.218750,4332.0,79.6875,46.46875,0,2.867188,1,0,6.0,65.0625,69271.531250,66.6875,58.21875
4,69.25,11.265625,9.062500,50.59375,5.109375,19.203125,4332.0,79.6875,46.46875,0,3.015625,1,0,6.0,65.0625,69198.703125,66.8125,58.12500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,69.25,0.562500,4.828125,63.90625,4.027344,19.656250,4900.0,67.5000,43.34375,1,15.867188,1,1,5.0,68.4375,71170.164062,71.7500,54.96875
1796,69.25,0.520996,4.843750,63.78125,4.000000,19.656250,5124.0,67.4375,43.37500,1,15.796875,1,1,5.0,68.5625,71460.804688,71.7500,54.93750
1797,69.25,0.569336,4.855469,63.93750,3.986328,19.640625,4900.0,67.5625,43.37500,1,15.765625,1,1,5.0,68.6875,71460.804688,71.7500,54.87500
1798,69.25,0.458252,4.871094,63.87500,3.945312,19.640625,4900.0,67.6875,43.40625,1,15.742188,1,1,5.0,68.7500,71405.906250,71.7500,54.90625


In [50]:
# 改变时间窗口后，这里要改动
df_days_past = df_valid[-SEQ_LEN:]
df_days_past.reset_index(drop=True,inplace=True)
df_days_past.info()
df_days_past.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   推料器自动指令    300 non-null    float16
 1   CO含量       300 non-null    float16
 2   HCL含量      300 non-null    float16
 3   NOx含量      300 non-null    float16
 4   SO2含量      300 non-null    float16
 5   二次风调门      300 non-null    float16
 6   二次风量       300 non-null    float16
 7   给水流量       300 non-null    float16
 8   炉排实际运行指令   300 non-null    float16
 9   炉排自动投退信号   300 non-null    int8   
 10  汽包水位       300 non-null    float16
 11  推料器启停      300 non-null    int8   
 12  推料器自动投退信号  300 non-null    int8   
 13  氧量设定值      300 non-null    float16
 14  一次风调门      300 non-null    float16
 15  一次风量       300 non-null    float32
 16  引风机转速      300 non-null    float16
 17  主蒸汽流量      300 non-null    float16
dtypes: float16(14), float32(1), int8(3)
memory usage: 10.4 KB


,推料器自动指令,CO含量,HCL含量,NOx含量,SO2含量,二次风调门,二次风量,给水流量,炉排实际运行指令,炉排自动投退信号,汽包水位,推料器启停,推料器自动投退信号,氧量设定值,一次风调门,一次风量,引风机转速,主蒸汽流量
0,56.5,-0.090088,12.359375,54.1250,42.28125,19.203125,5124.0,73.4375,46.46875,0,-6.828125,0,0,6.0,65.0625,68703.117188,68.3750,61.18750
1,56.5,-0.229004,12.390625,54.0625,42.31250,19.203125,5124.0,73.5000,46.46875,0,-6.992188,0,0,6.0,65.0625,68752.023438,68.3750,61.15625
2,56.5,-0.138672,12.390625,53.8750,42.25000,19.234375,5124.0,73.7500,46.46875,0,-7.285156,0,0,6.0,65.0625,68743.875000,68.4375,61.00000
3,56.5,-0.201172,12.367188,53.8750,42.25000,19.265625,5408.0,73.8750,46.46875,0,-7.492188,0,0,6.0,65.0625,68776.460938,68.3750,61.09375
4,56.5,-0.138672,12.335938,53.9375,42.21875,19.234375,5408.0,74.0625,46.46875,0,-7.519531,0,0,6.0,65.0625,68743.875000,68.3750,61.03125


In [51]:
df_days_future = df_test
df_days_future.info()
df_days_future['推料器自动指令'] = 0

df_days_future.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1800 entries, 0 to 1799
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   推料器自动指令    1800 non-null   float16
 1   CO含量       1800 non-null   float16
 2   HCL含量      1800 non-null   float16
 3   NOx含量      1800 non-null   float16
 4   SO2含量      1800 non-null   float16
 5   二次风调门      1800 non-null   float16
 6   二次风量       1800 non-null   float16
 7   给水流量       1800 non-null   float16
 8   炉排实际运行指令   1800 non-null   float16
 9   炉排自动投退信号   1800 non-null   int8   
 10  汽包水位       1800 non-null   float16
 11  推料器启停      1800 non-null   int8   
 12  推料器自动投退信号  1800 non-null   int8   
 13  氧量设定值      1800 non-null   float16
 14  一次风调门      1800 non-null   float16
 15  一次风量       1800 non-null   float32
 16  引风机转速      1800 non-null   float16
 17  主蒸汽流量      1800 non-null   float16
dtypes: float16(14), float32(1), int8(3)
memory usage: 75.6 KB


,推料器自动指令,CO含量,HCL含量,NOx含量,SO2含量,二次风调门,二次风量,给水流量,炉排实际运行指令,炉排自动投退信号,汽包水位,推料器启停,推料器自动投退信号,氧量设定值,一次风调门,一次风量,引风机转速,主蒸汽流量
0,0,11.078125,8.820312,50.34375,5.296875,19.234375,4900.0,79.6250,46.46875,0,2.798828,1,0,6.0,65.0625,68996.023438,66.5625,58.03125
1,0,11.031250,8.851562,50.34375,5.277344,19.234375,4664.0,79.6875,46.46875,0,2.857422,1,0,6.0,65.0625,68996.023438,66.5625,58.31250
2,0,11.039062,8.867188,50.46875,5.214844,19.203125,4664.0,79.7500,46.46875,0,2.867188,1,0,6.0,65.0625,69109.601562,66.6250,58.28125
3,0,11.085938,8.796875,50.59375,5.257812,19.218750,4332.0,79.6875,46.46875,0,2.867188,1,0,6.0,65.0625,69271.531250,66.6875,58.21875
4,0,11.265625,9.062500,50.59375,5.109375,19.203125,4332.0,79.6875,46.46875,0,3.015625,1,0,6.0,65.0625,69198.703125,66.8125,58.12500


In [52]:
# print('df_train.shape: ', df_train.shape)
# print('df_valid.shape: ', df_valid.shape)
# print('df_test.shape: ', df_test.shape)

In [53]:
old_scaled_array=scaler.transform(df_days_past)
new_scaled_array=scaler.transform(df_days_future)
new_scaled_df=pd.DataFrame(new_scaled_array)
print('new_scaled_array.shape: ', new_scaled_array.shape)
new_scaled_df.iloc[:,0]=np.nan
full_df=pd.concat([pd.DataFrame(old_scaled_array),new_scaled_df]).reset_index().drop(["index"],axis=1)
full_df.info()

new_scaled_array.shape:  (1800, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   推料器自动指令    300 non-null    float16
 1   CO含量       2100 non-null   float16
 2   HCL含量      2100 non-null   float16
 3   NOx含量      2100 non-null   float16
 4   SO2含量      2100 non-null   float16
 5   二次风调门      2100 non-null   float16
 6   二次风量       2100 non-null   float16
 7   给水流量       2100 non-null   float16
 8   炉排实际运行指令   2100 non-null   float16
 9   炉排自动投退信号   2100 non-null   int8   
 10  汽包水位       2100 non-null   float16
 11  推料器启停      2100 non-null   int8   
 12  推料器自动投退信号  2100 non-null   int8   
 13  氧量设定值      2100 non-null   float16
 14  一次风调门      2100 non-null   float16
 15  一次风量       2100 non-null   float32
 16  引风机转速      2100 non-null   float16
 17  主蒸汽流量      2100 non-null   float16
dtypes: float16(14), float32(1), int8(3)
memory usage: 71

In [54]:
full_df

,推料器自动指令,CO含量,HCL含量,NOx含量,SO2含量,二次风调门,二次风量,给水流量,炉排实际运行指令,炉排自动投退信号,汽包水位,推料器启停,推料器自动投退信号,氧量设定值,一次风调门,一次风量,引风机转速,主蒸汽流量
0,56.5,-0.090088,12.359375,54.12500,42.281250,19.203125,5124.0,73.4375,46.46875,0,-6.828125,0,0,6.0,65.0625,68703.117188,68.3750,61.18750
1,56.5,-0.229004,12.390625,54.06250,42.312500,19.203125,5124.0,73.5000,46.46875,0,-6.992188,0,0,6.0,65.0625,68752.023438,68.3750,61.15625
2,56.5,-0.138672,12.390625,53.87500,42.250000,19.234375,5124.0,73.7500,46.46875,0,-7.285156,0,0,6.0,65.0625,68743.875000,68.4375,61.00000
3,56.5,-0.201172,12.367188,53.87500,42.250000,19.265625,5408.0,73.8750,46.46875,0,-7.492188,0,0,6.0,65.0625,68776.460938,68.3750,61.09375
4,56.5,-0.138672,12.335938,53.93750,42.218750,19.234375,5408.0,74.0625,46.46875,0,-7.519531,0,0,6.0,65.0625,68743.875000,68.3750,61.03125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,NaN,0.562500,4.828125,63.90625,4.027344,19.656250,4900.0,67.5000,43.34375,1,15.867188,1,1,5.0,68.4375,71170.164062,71.7500,54.96875
2096,NaN,0.520996,4.843750,63.78125,4.000000,19.656250,5124.0,67.4375,43.37500,1,15.796875,1,1,5.0,68.5625,71460.804688,71.7500,54.93750
2097,NaN,0.569336,4.855469,63.93750,3.986328,19.640625,4900.0,67.5625,43.37500,1,15.765625,1,1,5.0,68.6875,71460.804688,71.7500,54.87500
2098,NaN,0.458252,4.871094,63.87500,3.945312,19.640625,4900.0,67.6875,43.40625,1,15.742188,1,1,5.0,68.7500,71405.906250,71.7500,54.90625


In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('use device: ', device)
model = Time_Model()


path = './save model/best_model.pt'
model.load_state_dict(torch.load(path))
model = model.to(device)


model.eval()

use device:  cuda:0


Time_Model(
  (lstm): LSTM(18, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [56]:
full_df_scaled_array=full_df.values
all_data=[]
time_step=SEQ_LEN
for i in range(time_step,len(full_df_scaled_array)):
    data_x=[]
    data_x.append(
    full_df_scaled_array[i-time_step :i , 0:full_df_scaled_array.shape[1]])
    data_x=np.array(data_x)

    data_x = torch.FloatTensor(data_x).to(device)
    with torch.no_grad():
        # print('-'*10)
        # print(data_x[0][0])
        prediction=model(data_x)
        # print(prediction)
        # print('-'*10)
    # print(prediction.shape)
    # break
    prediction = prediction.squeeze(-1)
    prediction = prediction.cpu().detach().numpy()

    all_data.append(prediction)
    full_df_scaled_array[i, 0] = prediction
    # full_df.iloc[i,0]=prediction

In [57]:
full_df_scaled_array[:, 0][-10:]

array([58.841137, 59.0605  , 58.654602, 59.029358, 58.868793, 58.788483,
       58.430443, 59.554134, 58.767166, 58.562542], dtype=float32)

In [58]:
# full_df_scaled_array[:, 0][-10:]

In [59]:
# all_data

In [60]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   推料器自动指令    300 non-null    float16
 1   CO含量       2100 non-null   float16
 2   HCL含量      2100 non-null   float16
 3   NOx含量      2100 non-null   float16
 4   SO2含量      2100 non-null   float16
 5   二次风调门      2100 non-null   float16
 6   二次风量       2100 non-null   float16
 7   给水流量       2100 non-null   float16
 8   炉排实际运行指令   2100 non-null   float16
 9   炉排自动投退信号   2100 non-null   int8   
 10  汽包水位       2100 non-null   float16
 11  推料器启停      2100 non-null   int8   
 12  推料器自动投退信号  2100 non-null   int8   
 13  氧量设定值      2100 non-null   float16
 14  一次风调门      2100 non-null   float16
 15  一次风量       2100 non-null   float32
 16  引风机转速      2100 non-null   float16
 17  主蒸汽流量      2100 non-null   float16
dtypes: float16(14), float32(1), int8(3)
memory usage: 71.9 KB


In [61]:
new_array=np.array(all_data)
new_array=new_array.reshape(-1,1)
prediction_copies_array = np.repeat(new_array,18, axis=-1)
y_pred_future_days = scaler.inverse_transform(np.reshape(prediction_copies_array,(len(new_array),18)))[:,0]
print(y_pred_future_days)

[58.44201  58.608704 58.556267 ... 59.554134 58.767166 58.562542]


In [62]:

len(all_data)

1800

In [63]:
prediction_copies_array


array([[58.44201 , 58.44201 , 58.44201 , ..., 58.44201 , 58.44201 ,
        58.44201 ],
       [58.608704, 58.608704, 58.608704, ..., 58.608704, 58.608704,
        58.608704],
       [58.556267, 58.556267, 58.556267, ..., 58.556267, 58.556267,
        58.556267],
       ...,
       [59.554134, 59.554134, 59.554134, ..., 59.554134, 59.554134,
        59.554134],
       [58.767166, 58.767166, 58.767166, ..., 58.767166, 58.767166,
        58.767166],
       [58.562542, 58.562542, 58.562542, ..., 58.562542, 58.562542,
        58.562542]], dtype=float32)

In [64]:
test = pd.read_csv('./data/test.csv')

In [65]:

# sub = pd.DataFrame({'ID': list(range(1,1801)),
#                     'Time': test['时间'],
#                     'Steam_flow': y_pred_future_days})
# 
# 
# sub.to_csv('result.csv',index=False)
sub = pd.DataFrame()
sub['ID'] = list(range(1,1801))
sub['Time'] = test['时间']
sub['Aps'] = test['推料器启停'].astype(bool)
sub['Ai'] = y_pred_future_days

sub.to_csv('./result.csv', index=False)

In [ ]:

sub = pd.read_csv('./demo2.csv')

In [ ]:
import matplotlib.pyplot as plt
train = pd.read_csv('data/train.csv')
t1 = train['主蒸汽流量'].tolist()
t2 = sub['Steam_flow'].tolist()
plt.figure(dpi=200)
plt.plot(list(range(len(t1))), t1)
plt.plot(list(range(len(t1), len(t1)+len(t2))), t2)
plt.savefig('./pred.jpg')